In [ ]:
!pip install tensorflow
!pip install transformers
!pip install sentencepiece



import random as r
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import TFCamembertForSequenceClassification, CamembertTokenizer, AutoConfig
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
# (for reproduciblity)
r.seed(0)
tf.random.set_seed(0)

training_data = 'https://raw.githubusercontent.com/LaCrazyTomato/Group-Project-DM-ML-2021/main/data/training_data.csv'

df = pd.read_csv(training_data, encoding='utf-8')


X = df['sentence'].values # to remove the index
y = df['difficulty'].values



X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=0)


# We need to encode output variable (since they are strings)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
#y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)



# We define the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
config = AutoConfig.from_pretrained('camembert-base')
config.num_labels = 6
config.hidden_dropout_prob = 0.1

model = TFCamembertForSequenceClassification.from_pretrained('camembert-base',config=config)
                                                             

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

Some layers of TFCamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import sklearn
  
# Tokenize
X_train = tokenizer(X_train.tolist(), padding="max_length", truncation=True, return_tensors='tf')
X_train = {x: X_train[x] for x in tokenizer.model_input_names}

X_test = tokenizer(X_test.tolist(), padding="max_length", truncation=True, return_tensors='tf')
X_test = {x: X_test[x] for x in tokenizer.model_input_names}


# We will use tensorflow to build and train our model

# SETTINGS
BATCH_SIZE = 4
EPOCHS = 4

# Create tensorflow dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(BATCH_SIZE)\
    .prefetch(tf.data.AUTOTUNE)


test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# Prepare the model (solver, metrics)
model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


# Train
es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
lr_cb = tf.keras.callbacks.ReduceLROnPlateau(patience=0, restore_best_weights=True, verbose=1, min_lr=1e-8)
csv_cb = tf.keras.callbacks.CSVLogger('history.csv')
cp_cb = tf.keras.callbacks.ModelCheckpoint('cp')


#model.fit(train_ds,validation_data=val_ds,epochs=EPOCHS,callbacks=[es_cb, lr_cb, csv_cb, cp_cb])
model.fit(train_ds,epochs=EPOCHS,callbacks=[es_cb, lr_cb, csv_cb, cp_cb])

# Save the tuned model
model.save_pretrained('trained_camembert')

Epoch 1/4
 921/1080 [========================>.....] - ETA: 1:10:55 - loss: 1.4206 - accuracy: 0.3906